## Notebook to create a Hybrid Hydrological Model

**Description**

The following notebook contains the code to create, train, validate and test a Hybrid Hydrological Model 
using a LSTM network plus a process based rainfall-runoff model. The code allows for the creation of single-basin 
models, but it is conceptualized to create regional models. The code is intended as an intial introduction to the topic, 
in which we prioritized interpretability over modularity.

The logic of the code is heavily based on [Neural Hydrology](https://doi.org/10.21105/joss.04050)[1]. For a more 
flexible, robust and modular implementation of deep learning method in hydrological modeling we advice the use of Neural 
Hydrology. 

**Authors:**
- Eduardo Acuna Espinoza (eduardo.espinoza@kit.edu)
- Ralf Loritz
- Manuel Álvarez Chaves

**References:**

[1]: "F. Kratzert, M. Gauch, G. Nearing and D. Klotz: NeuralHydrology -- A Python library for Deep Learning research in hydrology. Journal of Open Source Software, 7, 4050, doi: 10.21105/joss.04050, 2022"

In [ ]:
#Import necessary packages
import sys
import time
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from typing import List, Dict

sys.path.append("../aux_functions")
from hydrological_models import SHM as conceptual_model  # define the conceptual model!
from functions_datasets import CAMELS_GB as DataBase # define what you import as DataBase!
from functions_training import nse_basin_averaged
from functions_evaluation import nse
from functions_aux import create_folder, set_random_seed, write_report

Part 1. Initialize information

In [ ]:
# paths to access the information
path_entities = '../data/basin_id/basins_camels_gb_60.txt'
path_data = '../data/CAMELS_GB'

# dynamic forcings and target
dynamic_input = ['precipitation', 'peti', 'temperature']
conceptual_input = ['precipitation', 'peti', 'temperature']
target = ['discharge_spec']

# static attributes that will be used
static_input = ['area',
                'elev_mean',
                'dpsbar',
                'sand_perc',
                'silt_perc',
                'clay_perc',
                'porosity_hypres',
                'conductivity_hypres',
                'soil_depth_pelletier',
                'dwood_perc',
                'ewood_perc',
                'crop_perc',
                'urban_perc',
                'reservoir_cap',
                'p_mean',
                'pet_mean',
                'p_seasonality',
                'frac_snow',
                'high_prec_freq',
                'low_prec_freq',
                'high_prec_dur',
                'low_prec_dur']

# time periods
training_period = ['1980-10-01','1997-12-31']
validation_period = ['1975-10-01','1980-09-30']
testing_period = ['1998-01-01','2008-12-31']

network_params = {
  "input_size": len(dynamic_input) + len(static_input),
  "no_of_layers":1,  
  "seq_length": 180,
  "warmup_period": 365,
  "training_length": 365,
  "hidden_size": 64,
  "batch_size":10,
  "no_of_epochs": 5,             
  "drop_out": 0.4, 
  "learning_rate": 0.005,
  "adapt_learning_rate_epoch": 10,
  "adapt_gamma_learning_rate": 0.5,
  'set_forget_gate':3,
  "n_conceptual_models":1,
  "parameter_type": {'dd': 'dynamic', 'f_thr': 'dynamic', 'sumax': 'dynamic', 'beta': 'dynamic', 'perc': 'dynamic',
                     'kf': 'dynamic', 'ki': 'dynamic', 'kb': 'dynamic'}
                     }

# device
running_device = 'cpu' #cpu or gpu

# define seed
seed = 42

# Name of the folder where the results will be stored (the folder must be created before running
# the code)
path_save_folder = '../results/models/LSTM_SHM'

# colorblind friendly palette for plotting
color_palette = {'observed': '#1f78b4','LSTM': '#ff7f00'}

In [ ]:
# Create folder to store the results
create_folder(folder_path=path_save_folder)

Part 2. Class to create the dataset object used to manage the information

In [ ]:
class BaseDataset_Hybrid(Dataset):
    """Base data set class to load and preprocess data.

    Parameters
    ----------
    dynamic_input : List[str]
        name of variables used as dynamic series input in the lstm
    static_input : List[str]
        name of static inputs used as input in the lstm (e.g. catchment attributes)
    conceptual_input: List[str]
        name of variables used as dynamic series input in the conceptual model
    target: List[str]
        target variable(s)
    sequence_length: int
        sequence length used in the LSTM
    warmup_period: int
        number of time intervals before the output of the LSTM is used in the conceptual model
    time_period: List[str]
        initial and final date (e.g. ['1987-10-01','1999-09-30']) of the time period of interest 
    path_entities: str
        path to a txt file that contain the id of the entities (e.g. catchment`s ids) that will be analyzed
    path_data: str
        path to the folder were the data is stored
    path_addional features: str
        Optional parameter. Allows the option to add any arbitrary data that is not included in the standard data sets.
        Path to a pickle file (or list of paths for multiple files), containing a dictionary with each key corresponding 
        to one basin id and the value is a date-time indexed pandas DataFrame.      
    forcing: List[str]
        For CAMELS-US dataset we should specificy which forcing data will be used (e.g. daymet, maurer, ndlas, etc.)
    check_Nan: bool
        Boolean that indicate if one should check of NaN values while processing the data
    """

    #Function to initialize the data
    def __init__(self, 
                 dynamic_input: List[str],
                 static_input: List[str],
                 conceptual_input: List[str],
                 target: List[str], 
                 sequence_length: int,
                 warmup_period: int,
                 time_period: List[str],
                 path_entities: str,
                 path_data: str,
                 path_additional_features: str = '',
                 forcings: List[str] = [],
                 check_NaN:bool = True
                 ):

        # read and create variables
        self.time_period = time_period # time period that is being considered
        self.dynamic_input = dynamic_input  # dynamic forcings going as inputs of in the lstm
        self.conceptual_input = conceptual_input 
        self.target = target  # target variable
        self.sequence_length = sequence_length # sequence length
        self.warmup_period = warmup_period # warmup period

        entities_ids = np.loadtxt(path_entities, dtype='str').tolist() 
        # save the cathments as a list even if there is just one
        self.entities_ids = [entities_ids] if isinstance(entities_ids, str) else entities_ids # catchments

        self.sequence_data = {} # store information that will be used to run the lstm
        self.df_ts = {} # store processed dataframes for all basins
        self.scaler = {} # information to standardize the data 
        self.basin_std = {} # std of the target variable of each basin (can be used later in the loss function)
        self.valid_entities= []

        # process the attributes
        self.static_input = static_input # static attributes going as inputs to the lstm
        if static_input:
            self.df_attributes = self._load_attributes(path_data)

        # process additional features that will be included in the inputs (optional) ---
        if path_additional_features:
            self.additional_features = self._load_additional_features(path_additional_features)
        
        # This loop goes through all the catchments. For each catchment in creates an entry in the dictionary
        # self.sequence_data, where we will store the information that will be sent to the lstm
        for id in self.entities_ids:
            # load time series for specific catchment id
            df_ts = self._load_data(path_data=path_data, catch_id=id, forcings = forcings)
            # add additional features (optional)
            if path_additional_features:
                df_ts = pd.concat([df_ts, self.additional_features[id]], axis=1)
            
            # Defines the start date considering the offset due to sequence length. We want that, if possible, the start
            # date is the first date of prediction.
            start_date = pd.to_datetime(self.time_period[0],format="%Y-%m-%d")
            end_date = pd.to_datetime(self.time_period[1],format="%Y-%m-%d")
            freq = pd.infer_freq(df_ts.index)
            warmup_start_date = start_date - (self.sequence_length+self.warmup_period-1)*pd.tseries.frequencies.to_offset(freq)
            
            # filter dataframe for the period and variables of interest
            keep_columns = self.dynamic_input + list(set(self.conceptual_input) - set(self.dynamic_input)) + self.target
            df_ts = df_ts.loc[warmup_start_date:end_date, keep_columns]
            
            # store dataframe
            self.df_ts[id] = df_ts
            
            # create dictionary entry for the basin
            self.sequence_data[id] = {}

            # store the information of the basin in a nested dictionary
            self.sequence_data[id]['x_d'] = torch.tensor(df_ts.loc[:, self.dynamic_input].values, dtype=torch.float32)
            self.sequence_data[id]['x_c'] = torch.tensor(df_ts.loc[:, self.conceptual_input].values, dtype=torch.float32)
            self.sequence_data[id]['y'] = torch.tensor(df_ts.loc[:, self.target].values, dtype=torch.float32)
            if self.static_input:
                self.sequence_data[id]['x_s'] = torch.tensor(self.df_attributes.loc[id].values, dtype=torch.float32)
                    
    def __len__(self):
        return len(self.valid_entities)
    
    def __getitem__(self, id):
        """This function is used by PyTorch's dataloader to extract the information"""
        basin, i = self.valid_entities[id]

        # tensor of inputs LSTM
        x_lstm = self.sequence_data[basin]['x_d'][i, :]
        if self.static_input:
            x_s = self.sequence_data[basin]['x_s'].repeat(x_lstm.shape[0],1)
            x_lstm = torch.cat([x_lstm, x_s], dim=1)
        
        # tensor of inputs conceptual
        x_conceptual = self.sequence_data[basin]['x_c'][i[self.sequence_length-1:], :]
        
        # tensor of outputs
        y_obs = self.sequence_data[basin]['y'][i[self.sequence_length-1+self.warmup_period:], :]

        # optional also return the basin_std
        if self.basin_std:
            basin_std = self.basin_std[basin].repeat(y_obs.size(0)).unsqueeze(1)
            return x_lstm, x_conceptual, y_obs, basin_std
        else:
            return x_lstm, x_conceptual, y_obs
        

    def _load_attributes(self, path_data: str) -> pd.DataFrame:
        """Call the specific function that reads the static attributes information.
        
        Parameters
        ----------
        path_data : str
            path to the folder were the data is stored
            
        Returns
        -------
        df_attributes: pd.DataFrame
            Dataframe containing the attributes of interest for the catchments of interest
        """
        df_attributes = DataBase.read_attributes(path_data=path_data)
        df_attributes = df_attributes.loc[self.entities_ids, self.static_input]
        return df_attributes

    def _load_data(self, path_data: str, catch_id:str, forcings:List[str]) -> pd.DataFrame:
        """Call the specific function that reads a specific catchment timeseries into a dataframe.

        Parameters
        ----------
        path_data : str
            path to the folder were the data is stored.
        catch_id : str
            basin_id.
        forcings : str
            Can be e.g. 'daymet' or 'nldas', etc. Must match the folder names in the 'basin_mean_forcing' directory. 

        Returns
        -------
        df: pd.DataFrame
            Dataframe with the catchments` timeseries
        """
        df_ts = DataBase.read_data(path_data=path_data, catch_id=catch_id, forcings = forcings)
        return df_ts

    def _load_additional_features(self, path_additional_features: str) -> Dict[str, pd.DataFrame]:
        """Read pickle dictionary containing additional features.

        Parameters
        ----------
        path_additional_features : str
            Path to a pickle file (or list of paths for multiple files), containing a dictionary with each key 
            corresponding to one basin id and the value is a date-time indexed pandas DataFrame.   

        Returns
        -------
        additional_features: Dict[str, pd.DataFrame]
            Dictionary where each key is a basin and each value is a date-time indexed pandas DataFrame with the 
            additional features
        """
        
        with open(path_additional_features, "rb") as file:
            additional_features = pickle.load(file)
        return additional_features
  
    def calculate_basin_std(self):
        """Fill the self.basin_std dictionary with the standard deviation of the target variables for each basin"""
        for id, data in self.sequence_data.items():
            self.basin_std[id] = torch.tensor(np.nanstd(data['y'].numpy()), dtype=torch.float32)
    
    def calculate_global_statistics(self):
        """Fill the self.scalar dictionary 
        
        The function calculates the global mean and standard deviation of the dynamic inputs, target variables and 
        static attributes, and store the in a dictionary. It will be used later to standardize used in the LSTM. This
        function should be called only in training period. 
        """
        global_x = np.vstack([df.loc[:, self.dynamic_input].values for df in self.df_ts.values()])
        self.scaler['x_d_mean'] = torch.tensor(np.nanmean(global_x, axis=0), dtype=torch.float32)
        self.scaler['x_d_std'] = torch.tensor(np.nanstd(global_x, axis=0), dtype=torch.float32)
        del global_x

        if self.static_input:
            self.scaler['x_s_mean'] = torch.tensor(self.df_attributes.mean().values, dtype= torch.float32)
            self.scaler['x_s_std'] = torch.tensor(self.df_attributes.std().values, dtype= torch.float32)
    
    def standardize_data(self):
        """Standardize the data used in the LSTM. """
        for basin in self.sequence_data.values():
            # Standardize input
            basin['x_d'] = (basin['x_d'] - self.scaler['x_d_mean']) / self.scaler['x_d_std']
            if self.static_input:
                basin['x_s'] = (basin['x_s'] - self.scaler['x_s_mean']) / self.scaler['x_s_std']

    def generate_valid_entitites(self, training_length:int=0):
        """Generate the basin-time period combinations used for running the model 

        The function generate the basin_id-time period combinations that are used to run the model. When we are in
        training, we specify a training_length. In validation and testing this parameter can be omitted, and it will
        use the whole period as a valid entity
        
        Parameters
        ----------
        training_length:int
            Number of timesteps (e.g. days) used to calculate the loss in training
        """
        for key, df in self.df_ts.items():
            if training_length >0:
                time_cuts = list(range(self.sequence_length -1 + self.warmup_period, 
                                       len(df) + 1 - training_length, 
                                       training_length))
                # Create maps
                for cut in time_cuts:
                    self.valid_entities.append([key, list(range(cut - (self.sequence_length - 1 + self.warmup_period), 
                                                                cut  + training_length))])
                    
            else:
                self.valid_entities.append([key, list(range(0, len(df)))])

In [ ]:
# Class to create the hybrid model. Here is where I combine the LSTM network with the hydrological model
class Hybrid_Model(nn.Module):
    def __init__(self, network_params):
        super().__init__()
        # General information for the model
        self.seq_length = network_params['seq_length']

        self.num_features = network_params['input_size']
        self.hidden_units = network_params['hidden_size']
        self.num_layers = network_params['no_of_layers']

        # conceptual model
        self.conceptual_model = torch.jit.script(conceptual_model(n_models=network_params['n_conceptual_models'],
                                                                  parameter_type=network_params['parameter_type']))
        # linear layer
        self.linear = nn.Linear(in_features=network_params['hidden_size'], 
                                out_features=len(self.conceptual_model.parameter_ranges) * network_params['n_conceptual_models']) 
        # lstm
        self.lstm = nn.LSTM(input_size = network_params['input_size'], 
                            hidden_size = network_params['hidden_size'], 
                            batch_first = True,
                            num_layers = network_params['no_of_layers'])
           
    def forward(self, x_lstm, x_conceptual):
        
        # Initialize hidden state with zeros
        batch_size = x_lstm.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units, requires_grad=True, dtype=torch.float32, device=x_lstm.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units, requires_grad=True, dtype=torch.float32, device=x_lstm.device)
        
        # run LSTM 
        lstm_out, _ = self.lstm(x_lstm, (h0, c0))
        
        # map lstm outputs to the dimension of the conceptual model´s parameters
        lstm_out = lstm_out[:,self.seq_length-1:,:] # sequence to sequence
        lstm_out = self.linear(lstm_out)
        
        # get predictions
        pred = self.conceptual_model(x_conceptual=x_conceptual, lstm_out=lstm_out)

        return pred

Part 3. Create the different datasets

In [ ]:
# Dataset training
training_dataset = BaseDataset_Hybrid(dynamic_input=dynamic_input,
                                      static_input=static_input,
                                      conceptual_input=conceptual_input,
                                      target=target,
                                      sequence_length=network_params['seq_length'],
                                      warmup_period=network_params['warmup_period'],
                                      time_period=training_period,
                                      path_entities=path_entities,
                                      path_data=path_data,
                                      check_NaN=False)

training_dataset.calculate_basin_std()
training_dataset.calculate_global_statistics() # the global statistics are calculated in the training period!
training_dataset.standardize_data()
training_dataset.generate_valid_entitites(training_length=network_params["training_length"])

In [ ]:
# Dataset validation
validation_dataset = BaseDataset_Hybrid(dynamic_input=dynamic_input,
                                        static_input=static_input,
                                        conceptual_input=conceptual_input,
                                        target=target,
                                        sequence_length=network_params['seq_length'],
                                        warmup_period=network_params['warmup_period'],
                                        time_period=validation_period,
                                        path_entities=path_entities,
                                        path_data=path_data,
                                        check_NaN=False)

validation_dataset.scaler = training_dataset.scaler # read the global statisctics calculated in the training period
validation_dataset.standardize_data()
validation_dataset.generate_valid_entitites()

Part 4. Create the different dataloaders

In [ ]:
#Create DataLoader for training data.
train_loader = DataLoader(training_dataset, 
                          batch_size=network_params['batch_size'],
                          shuffle=True,
                          drop_last = True)

print('Batches in training: ', len(train_loader))
x_lstm, x_conceptual, y_obs, basin_std = next(iter(train_loader))
print(f'x_lstm: {x_lstm.shape} | x_conceptual: {x_conceptual.shape} | y_obs: {y_obs.shape} | basin_std: {basin_std.shape}')

In [ ]:
# DataLoader for validation data.
validation_loader = DataLoader(validation_dataset, 
                               batch_size=1,
                               shuffle=False,
                               drop_last = False)

# see if the batches are loaded correctly
print('Batches in validation: ', len(validation_loader))
x_lstm, x_conceptual, y_obs = next(iter(validation_loader))
print(f'x_lstm: {x_lstm.shape} | x_conceptual: {x_conceptual.shape} | y_obs: {y_obs.shape}')

# create some lists with the valid basins and the valid entities per basin that will help later to organize the data
valid_basins = [key for key, _ in validation_dataset.df_ts.items()]

Part 5. Define LSTM class

In [ ]:
# check if model will be run in gpu or cpu
if running_device == 'gpu':
    print(torch.cuda.get_device_name(0))
    device= f'cuda:0'
elif running_device == 'cpu':
    device = "cpu"

# construct model
set_random_seed(seed=seed)
hybrid_model = Hybrid_Model(network_params).to(device)

Part 6. Train LSTM

In [ ]:
# optimizer: Adam, learning rate, weight decay
optimizer = torch.optim.Adam(hybrid_model.parameters(),
                             lr = network_params["learning_rate"])
    
# define learning rate scheduler / multiple ways to adapt the learning rate
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            step_size = network_params["adapt_learning_rate_epoch"],
                                            gamma = network_params["adapt_gamma_learning_rate"])

# set forget gate to 3 to ensure that the model is capable to learn long term dependencies
hybrid_model.lstm.bias_hh_l0.data[network_params['hidden_size']:2 * network_params['hidden_size']] = network_params["set_forget_gate"]

training_time = time.time()
for epoch in range(network_params["no_of_epochs"]):

    epoch_start_time = time.time()
    total_loss = 0
    # Training ----------------------------------------------------------------------
    hybrid_model.train()
    for x_lstm, x_conceptual, y, basin_std in train_loader: 
        optimizer.zero_grad() # sets gradients of weigths and bias to zero
        pred = hybrid_model(x_lstm= x_lstm.to(device), 
                            x_conceptual= x_conceptual.to(device)) # forward call
        
        loss = nse_basin_averaged(y_sim = pred['y_hat'][:,training_dataset.warmup_period:], 
                                  y_obs = y.to(device),
                                  per_basin_target_std = basin_std.to(device)) 

        loss.backward() # backpropagates
        torch.nn.utils.clip_grad_norm_(hybrid_model.parameters(), 1) #clip gradients
        optimizer.step() #update weights
        total_loss += loss.item()
        
        # remove from cuda
        torch.cuda.empty_cache()
        del x_lstm, x_conceptual, y, pred
    
    # Average loss training
    average_loss_training = total_loss / len(train_loader)
    
   # validation ----------------------------------------------------------------------
    hybrid_model.eval()
    validation_results={}
    with torch.no_grad():
        for i, (x_lstm, x_conceptual, y) in enumerate(validation_loader):
            pred = hybrid_model(x_lstm= x_lstm.to(device), x_conceptual= x_conceptual.to(device)) # forward call

            # join results in a dataframe and store them in a dictionary (is easier to plot later)
            df_ts = validation_dataset.df_ts[valid_basins[i]].iloc[validation_dataset.sequence_length+
                                                                validation_dataset.warmup_period-1:]
            df_new = pd.DataFrame(data={'y_obs': y.flatten().cpu().detach().numpy(), 
                                        'y_sim': pred['y_hat'][:,validation_dataset.warmup_period:].flatten().cpu().detach().numpy()}, index=df_ts.index)
            df_ts = pd.concat([df_ts, df_new], axis=1)
            df_ts = df_ts.filter(['y_obs', 'y_sim'])
            validation_results[valid_basins[i]] = df_ts

            # remove from cuda
            del x_lstm, x_conceptual, y, pred
            torch.cuda.empty_cache()       
            
        #average loss validation
        loss_validation = nse(df_results=validation_results)
         
    # save model after every epoch
    path_saved_model = path_save_folder+'/epoch_' + str(epoch+1)
    torch.save(hybrid_model.state_dict(), path_saved_model)
            
    # print epoch report
    epoch_training_time = time.time()-epoch_start_time
    LR = optimizer.param_groups[0]['lr']
    report = f'Epoch: {epoch+1:<2} | Loss training: {"%.3f "% (average_loss_training)} | NSE validation: {"%.3f "% (loss_validation)} | LR:{"%.5f "% (LR)} | Training time: {"%.1f "% (epoch_training_time)} s'
    print(report)
    # save epoch report in txt file
    write_report(file_path=path_save_folder+'/run_progress.txt', text=report)
    # modify learning rate
    scheduler.step()

# print total report
total_training_time = time.time()-training_time
report = f'Total training time: {"%.1f "% (total_training_time)} s'
print(report)
# save total report in txt file
write_report(file_path=path_save_folder+'/run_progress.txt', text=report) 

Part 7. Test LSTM

In [ ]:
# In case I already trained an LSTM I can re-construct the model
#hybrid_model.load_state_dict(torch.load(path_save_folder + '/epoch_20'))

In [ ]:
# Dataset testing
test_dataset = BaseDataset_Hybrid(dynamic_input=dynamic_input,
                                  static_input=static_input,
                                  conceptual_input=conceptual_input,
                                  target=target,
                                  sequence_length=network_params['seq_length'],
                                  warmup_period=network_params['warmup_period'],
                                  time_period=testing_period,
                                  path_entities=path_entities,
                                  path_data=path_data,
                                  check_NaN=False)

test_dataset.scaler = training_dataset.scaler # read the global statisctics calculated in the training period
test_dataset.standardize_data()
test_dataset.generate_valid_entitites()


# DataLoader testing
test_loader = DataLoader(test_dataset, 
                         batch_size=1,
                         shuffle=False,
                         drop_last = False)

# see if the batches are loaded correctly
print('Batches in testing: ', len(test_loader))
x_lstm, x_conceptual, y_obs = next(iter(test_loader))
print(f'x_lstm: {x_lstm.shape} | x_conceptual: {x_conceptual.shape} | y_obs: {y_obs.shape}')

# create some lists with the valid basins and the valid entities per basin that will help later to organize the data
valid_basins_testing = [key for key, _ in test_dataset.df_ts.items()]

In [ ]:
# Testing ----------------------------------------------------------------------
hybrid_model.eval()
test_results={}
with torch.no_grad():
    for i, (x_lstm, x_conceptual, y) in enumerate(test_loader):
        
        pred = hybrid_model(x_lstm= x_lstm.to(device), x_conceptual= x_conceptual.to(device)) # forward call

        # join results in a dataframe and store them in a dictionary (is easier to plot later)
        df_ts = test_dataset.df_ts[valid_basins_testing[i]].iloc[test_dataset.sequence_length+ test_dataset.warmup_period-1:]
        
        # extract the discharge information
        df_discharge = pd.DataFrame(data={'y_obs': y.flatten().cpu().detach().numpy(), 
                                          'y_sim': pred['y_hat'][:, test_dataset.warmup_period:].flatten().cpu().detach().numpy()}, index=df_ts.index)
        df_discharge = pd.concat([df_ts, df_discharge], axis=1)
        df_discharge = df_discharge.filter(['y_obs', 'y_sim'])
        
        # extract internal_state (buckets) information
        internal_states = {key: value[0, test_dataset.warmup_period:, :].cpu().detach().numpy() for key, value in pred['internal_states'].items()}
        
        # extract parameter  information
        parameters = {key: value[0 , test_dataset.warmup_period:, :].cpu().detach().numpy() for key, value in pred['parameters'].items()}
        
        test_results[valid_basins_testing[i]] = {'discharges': df_discharge,
                                                 'internal_states': internal_states,
                                                 'parameters': parameters}

        # remove from cuda
        del x_lstm, x_conceptual, y, pred
        torch.cuda.empty_cache()

# Save results as a pickle file
#with open(path_save_folder+'/test_results.pickle', 'wb') as f:
 #   pickle.dump(test_results, f)

Part 8. Initial analysis of the results

In [ ]:
# Loss testing
discharges =  {key: value['discharges'] for key, value in test_results.items()}
loss_testing = nse(df_results=discharges, average=False)
df_NSE = pd.DataFrame(data={'basin_id': valid_basins_testing, 'NSE': np.round(loss_testing,3)})
df_NSE = df_NSE.set_index('basin_id')
#df_NSE.to_csv(path_save_folder+'/NSE.csv', index=True, header=True)

# Save simulated and observed discharges for all the basins in csv format
df_y_sim = pd.concat([pd.DataFrame({key: value['y_sim']}) for key, value in discharges.items()], axis=1)
df_y_obs = pd.concat([pd.DataFrame({key: value['y_obs']}) for key, value in discharges.items()], axis=1)
#df_y_sim.to_csv(path_save_folder+'/y_sim.csv', index=True, header=True)
#df_y_obs.to_csv(path_save_folder+'/y_obs.csv', index=True, header=True)

In [ ]:
# Plot the histogram
plt.hist(df_NSE['NSE'], bins=[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])

# Add NSE statistics to the plot
plt.text(0.01, 0.8, f'Mean: {"%.2f" % df_NSE["NSE"].mean():>7}\nMedian: {"%.2f" % df_NSE["NSE"].median():>0}\nMax: {"%.2f" % df_NSE["NSE"].max():>9}\nMin: {"%.2f" % df_NSE["NSE"].min():>10}',
         transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))

# Format plot
plt.rcParams["figure.figsize"] = (20, 5)
plt.xlabel('NSE', fontsize=12, fontweight='bold')
plt.ylabel('Frequency', fontsize=12, fontweight='bold')
plt.title('NSE Histogram', fontsize=16, fontweight='bold')
#plt.savefig(save_folder+'/NSE_Histogram.png', bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
# Plot simulated and observed discharges
basin_to_analyze = '2001'

plt.plot(df_y_obs[basin_to_analyze], label="observed", color=color_palette["observed"])
plt.plot(df_y_sim[basin_to_analyze], label="simulated", alpha=0.5, color=color_palette["LSTM"])

# Format plot
plt.xlabel('Day', fontsize=12, fontweight='bold')
plt.ylabel('Discharge [mm/d]', fontsize=12, fontweight='bold')
plt.title('Discharge comparison', fontsize=16, fontweight='bold')
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(loc="upper right",fontsize=12)
#plt.savefig(save_folder+'/Model_Comparison.png', bbox_inches='tight', pad_inches=0)

In [ ]:
# Plot states
state_of_interest = 'si'

states = test_results[basin_to_analyze]['internal_states'][state_of_interest]

for i in range(states.shape[1]):
    plt.plot(df_y_obs.index, states[:, i], label=state_of_interest+'_'+str(i+1))

# Adding labels and legend
plt.xlabel('Day', fontsize=12, fontweight='bold')
plt.ylabel('State [mm/d]', fontsize=12, fontweight='bold')
plt.title('Time evolution of internal states of conceptual model', fontsize=16, fontweight='bold')
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(loc="upper right",fontsize=12)
plt.show()

In [ ]:
# Plot states
parameter_of_interest= 'ki'

states = test_results[basin_to_analyze]['parameters'][parameter_of_interest]

for i in range(states.shape[1]):
    plt.plot( df_y_obs.index, states[:, i], label=parameter_of_interest+'_'+str(i+1), alpha=0.8)

# Adding labels and legend
plt.xlabel('Day', fontsize=12, fontweight='bold')
plt.ylabel('State [mm/d]', fontsize=12, fontweight='bold')
plt.title('Time evolution of parameters of conceptual model', fontsize=16, fontweight='bold')
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(loc="upper right",fontsize=12)
plt.show()